<a href="https://colab.research.google.com/github/pfflanders/Sign-Language-Translation/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip /content/gdrive/MyDrive/sign-language/data/live_color.zip

Streaming output truncated to the last 5000 lines.
  inflating: live_color/y/16384907902460_19.jpg  
  inflating: live_color/y/16384907902460_190.jpg  
  inflating: live_color/y/16384907902460_191.jpg  
  inflating: live_color/y/16384907902460_192.jpg  
  inflating: live_color/y/16384907902460_193.jpg  
  inflating: live_color/y/16384907902460_194.jpg  
  inflating: live_color/y/16384907902460_195.jpg  
  inflating: live_color/y/16384907902460_196.jpg  
  inflating: live_color/y/16384907902460_197.jpg  
  inflating: live_color/y/16384907902460_198.jpg  
  inflating: live_color/y/16384907902460_199.jpg  
  inflating: live_color/y/16384907902460_2.jpg  
  inflating: live_color/y/16384907902460_20.jpg  
  inflating: live_color/y/16384907902460_200.jpg  
  inflating: live_color/y/16384907902460_201.jpg  
  inflating: live_color/y/16384907902460_202.jpg  
  inflating: live_color/y/16384907902460_203.jpg  
  inflating: live_color/y/16384907902460_204.jpg  
  inflating: live_color/y/163849079

In [ ]:
import numpy as np 

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization

import os
import tensorflow as tf

from PIL import Image
import PIL

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Flatten, Dense, ReLU, Softmax, Conv2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.callbacks import TensorBoard, ReduceLROnPlateau, ModelCheckpoint

from keras.preprocessing.image import ImageDataGenerator

In [ ]:
IMG_SIZE = (56,56)
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.EfficientNetB4(input_shape=IMG_SHAPE, include_top=False, weights=None, pooling="avg")

model = Sequential([
    base_model,
    Flatten(),
    Dense(25),
    Softmax(),
])
model.compile(
    optimizer="adam",
    loss=CategoricalCrossentropy(),
    metrics=[CategoricalAccuracy()]
)
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb4 (Functional)  (None, 1792)             17673823  
                                                                 
 flatten (Flatten)           (None, 1792)              0         
                                                                 
 dense (Dense)               (None, 25)                44825     
                                                                 
 softmax (Softmax)           (None, 25)                0         
                                                                 
Total params: 17,718,648
Trainable params: 17,593,441
Non-trainable params: 125,207
_________________________________________________________________


In [ ]:
data_dir = "live_color"
target_size = (56, 56)
target_dims = (56, 56, 3) 
n_classes = 25
val_frac = 0.15
batch_size = 512

data_augmentor = ImageDataGenerator(
    horizontal_flip=True,
    rotation_range=20,
    validation_split=val_frac, 
    rescale=(1/255.0)
)

train_generator = data_augmentor.flow_from_directory(data_dir, target_size=target_size, batch_size=batch_size, shuffle=True, subset="training")
val_generator = data_augmentor.flow_from_directory(data_dir, target_size=target_size, batch_size=batch_size, shuffle=False, subset="validation")

Found 95359 images belonging to 25 classes.
Found 16827 images belonging to 25 classes.


In [ ]:
# import seaborn as sns
# model.load_weights(os.path.join("gdrive//MyDrive//sign-language//model_ckpts//mobilenet7"))
# x_pred = np.argmax(model.predict(train_generator), axis=-1)
# x_true = val_generator.classes
# matrix = tf.math.confusion_matrix(labels=x_true, predictions=x_pred).numpy()
# ax = sns.heatmap(matrix)

In [ ]:
training_run = 'efficientnetb4'
model.fit(
    train_generator, 
    callbacks=[
        ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=10),
        ModelCheckpoint(os.path.join("gdrive//MyDrive//sign-language//model_ckpts", training_run), save_best_only=True)
    ],
    epochs=30,
    validation_data=val_generator
)

Epoch 1/30
187/187 [==============================] - ETA: 0s - loss: 3.2418 - categorical_accuracy: 0.0571INFO:tensorflow:Assets written to: gdrive//MyDrive//sign-language//model_ckpts/efficientnetb2_2/assets


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


187/187 [==============================] - 474s 2s/step - loss: 3.2418 - categorical_accuracy: 0.0571 - val_loss: 3.2252 - val_categorical_accuracy: 0.0401 - lr: 0.0010
Epoch 2/30
187/187 [==============================] - 323s 2s/step - loss: 2.9591 - categorical_accuracy: 0.1223 - val_loss: 3.2475 - val_categorical_accuracy: 0.0401 - lr: 0.0010
Epoch 3/30
187/187 [==============================] - 323s 2s/step - loss: 2.1590 - categorical_accuracy: 0.3383 - val_loss: 4.0855 - val_categorical_accuracy: 0.0704 - lr: 0.0010
Epoch 4/30
187/187 [==============================] - ETA: 0s - loss: 1.2376 - categorical_accuracy: 0.6145INFO:tensorflow:Assets written to: gdrive//MyDrive//sign-language//model_ckpts/efficientnetb2_2/assets


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


187/187 [==============================] - 434s 2s/step - loss: 1.2376 - categorical_accuracy: 0.6145 - val_loss: 2.7640 - val_categorical_accuracy: 0.2645 - lr: 0.0010
Epoch 5/30
187/187 [==============================] - 326s 2s/step - loss: 0.6571 - categorical_accuracy: 0.7933 - val_loss: 3.2031 - val_categorical_accuracy: 0.3118 - lr: 0.0010
Epoch 6/30
187/187 [==============================] - 322s 2s/step - loss: 0.3937 - categorical_accuracy: 0.8762 - val_loss: 3.4603 - val_categorical_accuracy: 0.3385 - lr: 0.0010
Epoch 7/30
187/187 [==============================] - 323s 2s/step - loss: 0.2789 - categorical_accuracy: 0.9128 - val_loss: 4.2781 - val_categorical_accuracy: 0.2882 - lr: 0.0010
Epoch 8/30
187/187 [==============================] - 323s 2s/step - loss: 0.1881 - categorical_accuracy: 0.9392 - val_loss: 4.1360 - val_categorical_accuracy: 0.3490 - lr: 0.0010
Epoch 9/30
187/187 [==============================] - 322s 2s/step - loss: 0.1405 - categorical_accuracy: 0.953

In [ ]:
model.evaluate(val_generator)